
# FTC ML Workshop: From Data to Model (with TFLite)

This notebook is designed for **FTC teams** who want to understand **how Machine Learning works** instead of treating it as a black box. You'll:
- Explore and analyze data
- Train and evaluate models
- Understand common pitfalls (overfitting, data imbalance, leakage)
- Export a lightweight **TensorFlow Lite (TFLite)** model
- See how this maps to **FTC workflows** (VisionPortal + TFOD / FTC-ML)

> **Note**: FTC object recognition (boxes + labels) uses **object detection** models. Here, we start with **classification** (simpler) to build intuition, then show a **vision classifier** and **TFLite** conversion. You'll leave knowing the core ML process and how it translates to FTC workflows.



## 0) Setup
Run the next cell to import the libraries we need.


In [ ]:

import numpy as np
import pandas as pd

# Tabular ML
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Plotting
import matplotlib.pyplot as plt

# Vision ML
import tensorflow as tf
from tensorflow.keras import layers, models

# TFLite
from pathlib import Path

print("Versions:")
print("numpy", np.__version__)
print("pandas", pd.__version__)
import sklearn
print("scikit-learn", sklearn.__version__)
print("tensorflow", tf.__version__)



## 1) The ML Process (big picture)
1. **Define the problem**: What are we predicting and why?
2. **Collect data**: Images, sensor readings, etc.
3. **Label data** (for supervised learning): human-labeled classes/boxes.
4. **Split data**: Train vs. test (and sometimes validation).
5. **Train** a model on the train split.
6. **Evaluate** it on the test split using metrics appropriate for the task.
7. **Iterate**: Improve data quality, features, architecture, and hyperparameters.
8. **Export** a deployable artifact (e.g., `.tflite` for mobile/edge devices).
9. **Deploy & monitor**: Run it on-robot/in-app; keep notes for traceability & ethics.



## 2) Tabular Example (Iris dataset)
A quick win to see the end-to-end pipeline on a small dataset built into scikit-learn.


In [ ]:

# Load the Iris dataset
iris = load_iris(as_frame=True)
df = iris.frame.copy()
df.head()



### 2.1 Explore the data
Look at basic stats and relationships.


In [ ]:

df.describe()


In [ ]:

# Quick scatter plots (no specific colors set)
plt.figure()
plt.scatter(df['sepal length (cm)'], df['sepal width (cm)'])
plt.xlabel('sepal length (cm)')
plt.ylabel('sepal width (cm)')
plt.title('Sepal length vs width')
plt.show()

plt.figure()
plt.scatter(df['petal length (cm)'], df['petal width (cm)'])
plt.xlabel('petal length (cm)')
plt.ylabel('petal width (cm)')
plt.title('Petal length vs width')
plt.show()



### 2.2 Train / test split


In [ ]:

X = df[iris.feature_names].values
y = df['target'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train.shape, X_test.shape



### 2.3 Pipeline + model (Logistic Regression)
Standard practice: **scale → model** inside a `Pipeline`. Then evaluate.


In [ ]:

logreg_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=1000))
])

logreg_pipeline.fit(X_train, y_train)
y_pred = logreg_pipeline.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test accuracy (LogReg):", acc)

print("\nClassification report (LogReg):\n", classification_report(y_test, y_pred, target_names=iris.target_names))

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=iris.target_names)
disp.plot()
plt.title("Confusion Matrix (LogReg)")
plt.show()



### 2.4 Try a different model (Random Forest) + Cross-Validation


In [ ]:

rf_pipeline = Pipeline([
    ("clf", RandomForestClassifier(n_estimators=200, random_state=42))
])

rf_pipeline.fit(X_train, y_train)
y_pred_rf = rf_pipeline.predict(X_test)
acc_rf = accuracy_score(y_test, y_pred_rf)
print("Test accuracy (RF):", acc_rf)

cm_rf = confusion_matrix(y_test, y_pred_rf)
disp_rf = ConfusionMatrixDisplay(confusion_matrix=cm_rf, display_labels=iris.target_names)
disp_rf.plot()
plt.title("Confusion Matrix (Random Forest)")
plt.show()

scores = cross_val_score(rf_pipeline, X, y, cv=5)
print("5-fold CV accuracy (RF):", scores, "Mean:", scores.mean())



### 2.5 Takeaways (Tabular)
- Start simple; compare a couple of models.
- Use **train/test split** (and CV) to estimate generalization.
- Inspect **confusion matrix** to see where the model struggles.
- Document results for **traceability** (important for FTC ethics).



## 3) Vision Example (MNIST → TFLite)
Now let's switch to a **vision classifier** to see the Keras → TFLite path used for mobile/edge deployment.


In [ ]:

# Load MNIST
(mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = tf.keras.datasets.mnist.load_data()

# Normalize and add channel axis
mnist_x_train = mnist_x_train.astype("float32") / 255.0
mnist_x_test = mnist_x_test.astype("float32") / 255.0
mnist_x_train = np.expand_dims(mnist_x_train, -1)
mnist_x_test = np.expand_dims(mnist_x_test, -1)

print("Train:", mnist_x_train.shape, "Test:", mnist_x_test.shape)

# Build a small CNN
model = models.Sequential([
    layers.Input(shape=(28, 28, 1)),
    layers.Conv2D(16, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(10, activation="softmax"),
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(mnist_x_train, mnist_y_train, validation_split=0.1, epochs=3, batch_size=128, verbose=1)

test_loss, test_acc = model.evaluate(mnist_x_test, mnist_y_test, verbose=0)
print("MNIST test accuracy:", test_acc)



### 3.1 Visualize predictions


In [ ]:

# Show a few predictions
import random
indices = random.sample(range(len(mnist_x_test)), 9)
plt.figure()
for i, idx in enumerate(indices, start=1):
    plt.subplot(3, 3, i)
    plt.imshow(mnist_x_test[idx].squeeze(), cmap='gray')
    pred = np.argmax(model.predict(mnist_x_test[idx:idx+1], verbose=0))
    plt.title(f"Pred: {pred} / True: {mnist_y_test[idx]}")
    plt.axis('off')
plt.tight_layout()
plt.show()



### 3.2 Convert Keras model to **TFLite**
This mirrors how lightweight models are deployed on phones, Control Hubs, and embedded devices.


In [ ]:

from pathlib import Path
export_dir = Path('/mnt/data/ftc_ml_workshop')
export_dir.mkdir(parents=True, exist_ok=True)

# Save Keras model
keras_path = export_dir / 'mnist_cnn.h5'
model.save(keras_path)

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

tflite_path = export_dir / 'mnist_cnn.tflite'
with open(tflite_path, 'wb') as f:
    f.write(tflite_model)

print("Saved:", keras_path, "and", tflite_path, "Bytes:", len(tflite_model))



### 3.3 Run inference with the **TFLite Interpreter**


In [ ]:

# Use the TFLite interpreter to check that the model runs
interpreter = tf.lite.Interpreter(model_path=str(tflite_path))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Pick one sample
sample = mnist_x_test[:1]
interpreter.set_tensor(input_details[0]['index'], sample.astype(np.float32))
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])
pred = int(np.argmax(output))
true = int(mnist_y_test[0])
print("TFLite predicted:", pred, "| True:", true)



## 4) Mapping to FTC Workflows
- **Classification vs Object Detection**:  
  - *Classification* → one label per image (this notebook).  
  - *Object detection* → **bounding boxes + labels** (FTC TFOD).  
- **FTC-ML Toolchain**: Collect videos/images of game elements → label boxes → cloud-train → download `.tflite` → load in **VisionPortal + TFOD** (OnBot Java / Android Studio).
- **When to use which**:  
  - Use **FTC-ML** for game objects detection (easiest, officially supported).  
  - Use **custom training** (like Keras here) to learn concepts or build special-purpose models; convert to `.tflite` if deploying on device.



## 5) Ethics, Testing, and Traceability (DoD AI Principles, student-friendly)
- **Responsible**: Use and share data appropriately; no identifying faces without consent.
- **Equitable**: Avoid bias (e.g., only one lighting/background); diversify your data.
- **Traceable**: Keep a short training log (dates, dataset versions, parameters, metrics).
- **Reliable**: Test across conditions (lighting, angles, distance). Measure results.
- **Governable**: Have a safe fallback (manual control) and a way to disable the model.



## 6) Challenge Ideas for Students
- **Tabular**: Improve Iris accuracy using feature scaling, different models, or CV.
- **Vision**: Add simple **data augmentation** to the MNIST model (flips/rotations). Measure the effect.
- **Deployment**: Compare `.h5` vs `.tflite` file sizes; note the trade-offs.
- **Reporting**: Create a 1-page summary with **accuracy**, **confusion matrix**, and **lessons learned**.



### (Optional) Data augmentation for MNIST


In [ ]:

# Quick augmentation example: slight rotations and shifts
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1
)

aug_model = models.clone_model(model)
aug_model.set_weights(model.get_weights())
aug_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

history_aug = aug_model.fit(
    datagen.flow(mnist_x_train, mnist_y_train, batch_size=128),
    epochs=1,
    steps_per_epoch=len(mnist_x_train)//128,
    validation_data=(mnist_x_test, mnist_y_test),
    verbose=1
)

test_loss_aug, test_acc_aug = aug_model.evaluate(mnist_x_test, mnist_y_test, verbose=0)
print("MNIST test accuracy after 1 epoch of augmentation:", test_acc_aug)
